In [29]:
import pandas as pd
import numpy as np

In [30]:
# Load the dataset
file_path = "sample.csv"
df = pd.read_csv(file_path)
print("Original Data:\n", df.head())

Original Data:
        Id        Company               Model           Variant FuelType  \
0  555675  MARUTI SUZUKI  CELERIO(2017-2019)     1.0 ZXI AMT O   PETROL   
1  556383  MARUTI SUZUKI                ALTO               LXI   PETROL   
2  556422        HYUNDAI           GRAND I10    1.2 KAPPA ASTA   PETROL   
3  556771           TATA               NEXON           XT PLUS   PETROL   
4  559619           FORD                FIGO  EXI DURATORQ 1.4   DIESEL   

   Colour  Kilometer  BodyStyle TransmissionType ManufactureDate  ModelYear  \
0  Silver      33197  HATCHBACK              NaN      01-02-2018       2018   
1     Red      10322  HATCHBACK           Manual      01-03-2021       2021   
2    Grey      37889  HATCHBACK           Manual      01-03-2015       2015   
3    Blue      13106  HATCHBACK              NaN      01-08-2020       2020   
4  Silver     104614  HATCHBACK           Manual      01-11-2010       2010   

   CngKit       Price      Owner DealerState              

In [31]:
# Step 1: Handle Missing Values
# Fill missing FuelType with the most frequent value
if 'FuelType' in df.columns:
    df['FuelType'].fillna(df['FuelType'].mode()[0])
print("\nAfter Filling Missing FuelType:\n", df[['FuelType']].head())

# Fill missing TransmissionType based on most common value per Model (if possible)
if 'TransmissionType' in df.columns and 'Model' in df.columns:
    df['TransmissionType'] = df.groupby('Model')['TransmissionType'].transform(lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x)
print("\nAfter Filling Missing TransmissionType:\n", df[['TransmissionType']].head())

# Convert CngKit to a binary column if it exists
if 'CngKit' in df.columns:
    df['HasCNG'] = df['CngKit'].notna().astype(int)
    df.drop(columns=['CngKit'])
print("\nAfter Converting CngKit to Binary:\n", df[['HasCNG']].head())


After Filling Missing FuelType:
   FuelType
0   PETROL
1   PETROL
2   PETROL
3   PETROL
4   DIESEL

After Filling Missing TransmissionType:
   TransmissionType
0              NaN
1           Manual
2           Manual
3              NaN
4           Manual

After Converting CngKit to Binary:
    HasCNG
0       0
1       0
2       0
3       0
4       0


In [32]:
# Step 2: Convert Data Types
# Convert ManufactureDate to datetime format if it exists
if 'ManufactureDate' in df.columns:
    df['ManufactureDate'] = pd.to_datetime(df['ManufactureDate'], errors='coerce')
print("\nAfter Converting ManufactureDate to datetime:\n", df[['ManufactureDate']].head())

# Convert Price from 'Lakhs' format to numeric if it exists
if 'Price' in df.columns and df['Price'].dtype == object:
    df['Price'] = df['Price'].str.replace(' Lakhs', '', regex=False).astype(float) * 100000
print("\nAfter Converting Price to Numeric:\n", df[['Price']].head())

# Convert Owner column to numerical form if it exists
if 'Owner' in df.columns and df['Owner'].dtype == object:
    df['Owner'] = df['Owner'].str.extract(r'(\d+)').astype(float)
print("\nAfter Converting Owner to Numeric:\n", df[['Owner']].head())


After Converting ManufactureDate to datetime:
   ManufactureDate
0      2018-01-02
1      2021-01-03
2      2015-01-03
3      2020-01-08
4      2010-01-11

After Converting Price to Numeric:
       Price
0  575000.0
1  435000.0
2  470000.0
3  990000.0
4  270000.0

After Converting Owner to Numeric:
    Owner
0    1.0
1    1.0
2    1.0
3    1.0
4    2.0


In [33]:
# Step 3: Standardization of Categorical Data
# Normalize Company names to proper case if it exists
if 'Company' in df.columns:
    df['Company'] = df['Company'].str.title().str.strip()
print("\nAfter Standardizing Company Names:\n", df[['Company']].head())

# Standardize Colour names if it exists
if 'Colour' in df.columns:
    df['Colour'] = df['Colour'].str.strip().str.capitalize()
print("\nAfter Standardizing Colour Names:\n", df[['Colour']].head())


After Standardizing Company Names:
          Company
0  Maruti Suzuki
1  Maruti Suzuki
2        Hyundai
3           Tata
4           Ford

After Standardizing Colour Names:
    Colour
0  Silver
1     Red
2    Grey
3    Blue
4  Silver


In [34]:
# Step 4: Remove Duplicates
if {'Model', 'Variant', 'Kilometer', 'Price'}.issubset(df.columns):
    df.drop_duplicates(subset=['Model', 'Variant', 'Kilometer', 'Price'], keep='first')
print("\nAfter Removing Duplicates:\n", df.head())


After Removing Duplicates:
        Id        Company               Model           Variant FuelType  \
0  555675  Maruti Suzuki  CELERIO(2017-2019)     1.0 ZXI AMT O   PETROL   
1  556383  Maruti Suzuki                ALTO               LXI   PETROL   
2  556422        Hyundai           GRAND I10    1.2 KAPPA ASTA   PETROL   
3  556771           Tata               NEXON           XT PLUS   PETROL   
4  559619           Ford                FIGO  EXI DURATORQ 1.4   DIESEL   

   Colour  Kilometer  BodyStyle TransmissionType ManufactureDate  ModelYear  \
0  Silver      33197  HATCHBACK              NaN      2018-01-02       2018   
1     Red      10322  HATCHBACK           Manual      2021-01-03       2021   
2    Grey      37889  HATCHBACK           Manual      2015-01-03       2015   
3    Blue      13106  HATCHBACK              NaN      2020-01-08       2020   
4  Silver     104614  HATCHBACK           Manual      2010-01-11       2010   

   CngKit     Price  Owner DealerState       

In [35]:
# Step 5: Ensure Numerical Consistency
if 'Kilometer' in df.columns:
    df = df[(df['Kilometer'] > 0) & (df['Kilometer'] < 1_000_000)]
if 'Price' in df.columns:
    df = df[(df['Price'] > 10_000) & (df['Price'] < 10_000_000)]
if 'Warranty' in df.columns:
    df = df[df['Warranty'] >= 0]
print("\nAfter Ensuring Numerical Consistency:\n", df.head())


After Ensuring Numerical Consistency:
        Id        Company               Model           Variant FuelType  \
0  555675  Maruti Suzuki  CELERIO(2017-2019)     1.0 ZXI AMT O   PETROL   
1  556383  Maruti Suzuki                ALTO               LXI   PETROL   
2  556422        Hyundai           GRAND I10    1.2 KAPPA ASTA   PETROL   
3  556771           Tata               NEXON           XT PLUS   PETROL   
4  559619           Ford                FIGO  EXI DURATORQ 1.4   DIESEL   

   Colour  Kilometer  BodyStyle TransmissionType ManufactureDate  ModelYear  \
0  Silver      33197  HATCHBACK              NaN      2018-01-02       2018   
1     Red      10322  HATCHBACK           Manual      2021-01-03       2021   
2    Grey      37889  HATCHBACK           Manual      2015-01-03       2015   
3    Blue      13106  HATCHBACK              NaN      2020-01-08       2020   
4  Silver     104614  HATCHBACK           Manual      2010-01-11       2010   

   CngKit     Price  Owner DealerS

In [37]:
# Step 6: Check for Outliers using Interquartile Range (IQR) method
if 'Price' in df.columns:
     Q1 = df['Price'].quantile(0.25)
     Q3 = df['Price'].quantile(0.75)
     IQR = Q3 - Q1
     lower_bound = Q1 - 1.5 * IQR
     upper_bound = Q3 + 1.5 * IQR
     df = df[(df['Price'] >= lower_bound) & (df['Price'] <= upper_bound)]
print("\nAfter Removing Outliers:\n", df.head())


After Removing Outliers:
        Id        Company               Model           Variant FuelType  \
0  555675  Maruti Suzuki  CELERIO(2017-2019)     1.0 ZXI AMT O   PETROL   
1  556383  Maruti Suzuki                ALTO               LXI   PETROL   
2  556422        Hyundai           GRAND I10    1.2 KAPPA ASTA   PETROL   
3  556771           Tata               NEXON           XT PLUS   PETROL   
4  559619           Ford                FIGO  EXI DURATORQ 1.4   DIESEL   

   Colour  Kilometer  BodyStyle TransmissionType ManufactureDate  ModelYear  \
0  Silver      33197  HATCHBACK              NaN      2018-01-02       2018   
1     Red      10322  HATCHBACK           Manual      2021-01-03       2021   
2    Grey      37889  HATCHBACK           Manual      2015-01-03       2015   
3    Blue      13106  HATCHBACK              NaN      2020-01-08       2020   
4  Silver     104614  HATCHBACK           Manual      2010-01-11       2010   

   CngKit     Price  Owner DealerState         

In [39]:
# Save cleaned dataset
df.to_csv("C:/Users/Admin/Desktop/tybsc D98/Cars-Recommendation-System/cleaned_usedCars.csv", index=False)
print("\nData cleaning complete. Cleaned file saved as 'cleaned_usedCars.csv'.")


Data cleaning complete. Cleaned file saved as 'cleaned_usedCars.csv'.
